### Q10
Solution from Chaolun


#### 1)

Define the input $x=[x_1, x_2]$, the output $y$, the activation of hidden layer $h=[h_1, h_2, h_3]$ the weight matrix $W^0, W^1$, where $W_{ij}$ denotes the weight of directed edge from the i th node of the former layer to the j th node of the later layer. $\sigma^0()$ and $\sigma^1()$ are the elementary wise activation functions for the hidden layer and output layer separately.

We can have:
$$
h=\sigma^0(xW^0)
$$
and
$$
y=\sigma^1(hW^1)
$$
Given that $\sigma^0(x)=\sigma^1(x)=x$, we have:
$$
y=xW^0W^1
$$
Given that $\Theta(\sigma^0(x))=\Theta(\sigma^1(x))=1$, we have:

The operation count for calculating hidden layer=6(multiplications)+3(additions)+3($\sigma^0()$evaluation)=12

The operation count for calculating output layer=3(multiplications)+2(additions)+1($\sigma^1()$evaluation)=6

So intotal there are 18 operations.



#### 2)

Define the input $x=[x_1, x_2]$, the output $y$, the activation of hidden layer $h^i=[h^i_1, h^i_2...,h^i_j... h^i_m]$ , where $i=1,2,...,n$, the weight matrix $W^k$ ($k=0, 1,2,...,n$), where $W^k_{pq}$ denotes the weight of directed edge from the p th node of the former layer to the q th node of the later layer. $\sigma^k()$ are the elementary wise activation functions for the hidden layer and output layer neurons.

We can have:
$$
h^1=\sigma^0(xW^0)
$$
$$
h^k=\sigma^{k-1}(h^{k-1}W^{k-1})
$$
where {k=2,3,4....,n}

and
$$
y=\sigma^n(h^nW^n)
$$
Given that $\sigma^k(x)=x$, we have:
$$
y=xW^0W^1W^2....W^n
$$
Given that $\Theta(\sigma^k(x))=1$, we have:

The operation count for calculating first hidden layer=2m(multiplications)+m(additions)+m($\sigma^0()$evaluation)=4m

The operation count for calculating hidden layer 2 to n=(n-1)*($m^2$(multiplications)+m(m-1)(additions)+m($\sigma^k()$evaluation))=$2m^2(n-1)$

The operation count for calculating output layer=m(multiplications)+(m-1)(additions)+1($\sigma^1()$evaluation)=2m

So intotal there are $2m^2(n-1)+6m$ operations.

The theoretical time scaling for m is $O(m^2)$, the time scaling for n is $O(n)$

Here is the implementation:

In [46]:
import numpy as np
import functools as ft
from timeit import default_timer as timer

def weight(size):
    return np.random.uniform(-1,1, size)

# initialization
x=np.random.uniform(0,1,[1,2])

for m in [5,10]:
    for n in [5,10,15]:
        #bulid computational graph
        computational_graph=[]
        computational_graph.append(x)
        computational_graph.append(weight([2,m]))
        for i in range(n-1):
            computational_graph.append(weight([m,m]))
        computational_graph.append(weight([m,1]))

        #calculation ####done multiple times for accurate timing
        start=timer()
        for i in range(10000):
            y=ft.reduce(np.dot, computational_graph)
        end=timer()
        print("when m=%d, n=%d, time consumed: %f, the output value is %f\n"%(m, n, (end-start)/10000, np.float64(y)))


when m=5, n=5, time consumed: 0.000005, the output value is -0.350934

when m=5, n=10, time consumed: 0.000009, the output value is 12.799468

when m=5, n=15, time consumed: 0.000011, the output value is -1.213864

when m=10, n=5, time consumed: 0.000006, the output value is -4.073814

when m=10, n=10, time consumed: 0.000008, the output value is -139.333936

when m=10, n=15, time consumed: 0.000012, the output value is 1761.876788



The time takes to compute the computational graph grows linearly as n increase. However, the increasing of m will not lead to increasing to the computational time. Considering that theoretical time scaling for m is $O(m^2)$, the result is different as expected. This is becasue the implicit parallelization of numpy library, which support parrallel by vectorization.
 

#### 3)
As long as the activation function $\sigma^k(x)$ is defined as elementary wise operation on matrix x, the matrix formulation still work as following:

Define the input $x=[x_1, x_2]$, the output $y$, the activation of hidden layer $h^i=[h^i_1, h^i_2...,h^i_j... h^i_m]$ , where $i=1,2,...,n$, the weight matrix $W^k$ ($k=0, 1,2,...,n$), where $W^k_{pq}$ denotes the weight of directed edge from the p th node of the former layer to the q th node of the later layer. $\sigma^k()$ are the elementary wise activation functions for the hidden layer and output layer neurons.

We can have:
$$
h^1=\sigma^0(xW^0)
$$
$$
h^k=\sigma^{k-1}(h^{k-1}W^{k-1})
$$
where {k=2,3,4....,n}

and
$$
y=\sigma^n(h^nW^n)
$$
Given that $\sigma^k(x)=tanh(x)$, we have:
$$
y=tanh(...tanh(tanh(xW^0)W^1)....W^n)
$$
revised code:


In [49]:
import numpy as np
import functools as ft
from timeit import default_timer as timer

def weight(size):
    return np.random.uniform(-1,1, size)

def activation(x,y):
    return np.tanh(np.dot(x,y))

# initialization
x=np.random.uniform(0,1,[1,2])

for m in [5,10]:
    for n in [5,10,15]:
        #bulid computational graph
        computational_graph=[]
        computational_graph.append(x)
        computational_graph.append(weight([2,m]))
        for i in range(n-1):
            computational_graph.append(weight([m,m]))
        computational_graph.append(weight([m,1]))

        #calculation ####done multiple times for accurate timing
        start=timer()
        for i in range(10000):
            y=ft.reduce(activation, computational_graph)
        end=timer()
        print("when m=%d, n=%d, time consumed: %f, the output value is %f\n"%(m, n, (end-start)/10000, np.float64(y)))

when m=5, n=5, time consumed: 0.000015, the output value is -0.020980

when m=5, n=10, time consumed: 0.000022, the output value is -0.212521

when m=5, n=15, time consumed: 0.000026, the output value is -0.441610

when m=10, n=5, time consumed: 0.000010, the output value is -0.383114

when m=10, n=10, time consumed: 0.000019, the output value is -0.866461

when m=10, n=15, time consumed: 0.000028, the output value is 0.961389



#### 4)
The apporpriate solution can depends on two factors:

1. how many neurons are in radius R of another neuron.

2. the temporal locality of each neurons(do they only move a short distance(smaller than R) or they can jump to a random place for each consecutive evaluation).

If there is a relatively small number of neurons in radius R and the neurons have a good temporal locality, I would suggest using a lookup table, which keeps a list of neurons within radius R for each of the neurons. Since the neurons will not move quite long distance, the large porpotional of the list remains unchanged. In order to update the list(L) for one neuron(A), firstly check if the neighboring neurons in the list L are still in the radius R, delete if not, then delet A from the neighboring list of the deleted neurons. Then perform a BFS using the neuron A as root for 2 steps(with neighboring neurons in L as child), which generate a tree of 3 level. Check if the leafs of the tree is in the radius R, if true add the positive leaf into list L, and add A into the neighboring list of the positive leafs. The complexity of this algorithm is constant($O(1)$).

If the condition not meet and there is no limitation on how far the neuron can move for each step(which means BFS cannot find all the new neighbor for 2 steps), we can use the Quad tree to help finding the neighbors of a given neuron(a), which have $O(log(n))$ complexity. Here is an example of c++ implementation from https://github.com/bnpfeife/quadtree

#include <vector>

namespace bnp {

// When storing elements in the QuadTree, users may want to use unique/custom

// objects. To aid the QuadTree with interpreting different types of data,

// implement the qt_point_adapter on your objects.

class qt_point_adapter {

public:

    virtual float get_x() const = 0; // Retrieves x position
    
    virtual float get_y() const = 0; // Retrieves y position
    
    // Determines if point equals another
    
    virtual bool equals(const qt_point_adapter & point) const;
    
};

// This is used as the QuadTree's internal point structure. This prevents the

// QuadTree from explicitly using new/delete on points. If one wishes to use

// their own point objects, create an adapter function or use

// qt_point_adapter where applicable.

class qt_point : public qt_point_adapter {

private:

    float mX; // Stores x position
    
    float mY; // Stores y position
    
public:

    void set_x(const float& x); // Assigns x position
    
    void set_y(const float& y); // Assigns y position
    
    float get_x() const; // Retrieves x position
    
    float get_y() const; // Retrieves y position
    
    // Constructor for initializing POD members
    
    qt_point();
    
    // Constructor for assigning POD members
    
    qt_point(const float& x, const float& y);
    
};

// This is used as the QuadTree's internal boundary structure.

class qt_box {

private:

    float mX0; // Stores x0 position
    
    float mY0; // Stores y0 position
    
    float mX1; // Stores x1 position
    
    float mY1; // Stores y1 position
    

public:

    void set_x0(const float& x0); // Assigns x0 position
    
    void set_y0(const float& y0); // Assigns y0 position
    
    void set_x1(const float& x1); // Assigns x1 position
    
    void set_y1(const float& y1); // Assigns y1 position
    
    virtual float get_x0() const; // Retrieves x0 position
    
    virtual float get_y0() const; // Retrieves y0 position
    
    virtual float get_x1() const; // Retrieves x1 position
    
    virtual float get_y1() const; // Retrieves y1 position
    
    // Determines if qt_point or qt_box intersects
    
    bool intersects(const qt_point_adapter& point) const;
    
    bool intersects(const qt_box& box) const;
    

    // Constructor for initializing POD members
    
    qt_box();
    
    // Constructor for assigning POD members
    
    qt_box(const float& x0, const float& y0,
    
           const float& x1, const float& x2);
           
};

class quadtree {

private:

    quadtree* mNodeNW; // Stores north west node
    
    quadtree* mNodeNE; // Stores north east node
    
    quadtree* mNodeSW; // Stores south west node
    
    quadtree* mNodeSE; // Stores south east node
    
    qt_box mBounds; // Stores quadrant boundaries

    // Elements stored in the QuadTree are not owned by the QuadTree. This
    
    // allows the user access to the elements outside the QuadTree and have
    
    // them spatially organized within.
    
    std::vector<qt_point_adapter*> mElements;

    // Subdivides the QuadTree. This private because certain conditions must be
    
    // met before the tree can subdivide. In this implementation the QuadTree
    
    // subdivides when a new point is inserted, but does not match the point
    
    // location in the current node.
    
    void subdivide();
    
    // Joins the QuadTree. This does the opposite operation of subdivide. This
    
    // is private because certain conditions must be met before the QuadTree can
    
    // join. The QuadTree nodes must have no elements and must be joined.
    
    void join();

public:

    // Constructs an empty QuadTree. All QuadTrees must have specified
    
    // boundaries. All child QuadTrees are unaware that they are children and
    
    // have a parent tree. This is to maintain autonomy of the trees.
    
    quadtree(const qt_box& bounds);
    
    // QuadTree needs an explicit destructor. This is so that it can delete
    
    // it's children.
    
    ~quadtree();
    
    // QuadTree needs an explicit copy constructor and operator. C++'s default
    
    // copy will incorrectly link the subtrees to the new QuadTree. This
    
    // overrides this behavior.
    
    quadtree(const quadtree& qt);
    
    quadtree& operator=(const quadtree& qt);
    

    bool insert(qt_point_adapter* point); // Inserts point into tree
    
    bool remove(qt_point_adapter* point); // Removes point from tree
    
    // Retrieves elements from a defined region. If a node intersects the
    
    // defined region, elements are not automatically returned. Those elements
    
    // must also intersect the defined region.
    
    std::vector<qt_point_adapter*> query(const qt_box& bounds) const;
    
    std::vector<qt_point_adapter*> query(const qt_point& point) const;
    
    // Retrieves elements from a defined region and removes those elements from
    
    // the QuadTree. If a node intersects the defined region, elements are not
    
    // automatically returned. Those elements must also
    
    // intersect the defined region.
    
    std::vector<qt_point_adapter*> pull(const qt_box& bounds);
    
    std::vector<qt_point_adapter*> pull(const qt_point& point);
    
    // Clears the QuadTree and all of its children.
    
    void clear();
    
};

} // namespace bnp